In [ ]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
import numpy as np
import SimpleITK as sitk
import csv
import shutil

### Script to write csv_file with every nifti for DeepOncology repertory

#### - Read JSON dataset 

In [ ]:
json_path = '/media/deeplearning/Elements/REMARC_Validated_DICOMS/validated_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

#### - Get the list of every nifti in the directory 

In [ ]:
nifti_directory = '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/REMARC_NIFTI'
liste = os.listdir(nifti_directory)
liste_nifti = []
for nifti in liste : 
    liste_nifti.append(os.path.join(nifti_directory, nifti))

#### - Add nifti PT CT MASK for every data/study in dataset 

In [ ]:
for data in dataset : 
    uid = data[4]
    subliste = []
    for nifti in liste_nifti : 
        if uid in nifti : 
            subliste.append(nifti)
            subliste.sort()
            
    for i in range(3):
        data.append(subliste[i])
    

#### - Check if CT/PET/MASK nifti for every study 
#### if not, remove study from dataset 

In [ ]:
for serie in dataset : 
    if len(serie) !=  : 
        print(serie)

In [ ]:
print("nombre de study : ", len(dataset))

#### - Create folder with patient_id and put nifti into it 

In [ ]:
for serie in dataset: 
        #patient_name = serie[0]
    patient_id = serie[6]
    pet_type = "pet0"
    nifti_ct = serie[8]
    nifti_pet = serie[9]
    nifti_mask = serie[10] #if pet0


    folder = '/media/deeplearning/Elements/AHL_NIFTI'+'/'+patient_id

    if not os.path.exists(folder):
        os.makedirs(folder)
        print(folder, 'created')
    else : print(folder, "Dossier patient déjà crée")

    subfolder = folder+'/'+pet_type
    if not os.path.exists(subfolder) :
        os.makedirs(subfolder)
        print(subfolder, 'created')
    else : print(subfolder, "Déjà creer => already folder for ", pet_type)

    ct_path = nifti_ct
    pt_path = nifti_pet
    mask_path = nifti_mask

    new_ct_path = subfolder+'/'+nifti_ct.split('/')[-1]
    new_pet_path = subfolder+'/'+nifti_pet.split('/')[-1]
    new_mask_path = subfolder+'/'+nifti_mask.split('/')[-1]
    try : 

        os.rename(ct_path, new_ct_path)
        os.rename(pt_path, new_pet_path)
        os.rename(mask_path, new_mask_path)
            

        serie[8] = new_ct_path
        serie[9] = new_pet_path 
        serie[10] = new_mask_path

    except Exception as err : 
        print(err)



#### - Write csv_file 

In [ ]:
nifti_directory = '/media/deeplearning/Elements/AHL_NIFTI'
filename = 'AHL2011_NIFTI_V4.csv'

with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT NAME", "PATIENT ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK", "NIFTI_MERGED"])
    for serie in data : 
        csv_writer.writerow(["", serie[6], serie[4], 'pet0', serie[7], serie[8], serie[9]])
